# **Deep Learning Project**
Done by: <br>
- Ana Farinha | 20211514
- Ana Reis | 20211631
- Beatriz Barreto | 20211547
- Mariana Neto | 20211527


### <font color='#b185db'> Mount drive and import libraries</font>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import kerastuner as kt


from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
!pip install keras-tuner
!pip install keras-tuner --upgrade
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.2 MB/s eta 0:00:00


<ipython-input-2-bfac5854bd1d>:18: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### <font color='#b185db'> Import dataset</font>

In [ ]:
metadata_df = pd.read_csv("/content/drive/MyDrive/ProjectDL23_24/HAM10000_metadata.csv")
metadata_df = metadata_df.set_index('image_id')
metadata_df

,lesion_id,dx,dx_type,age,sex,localization
image_id,,,,,,
ISIC_0027419,HAM_0000118,bkl,histo,80.0,male,scalp
ISIC_0025030,HAM_0000118,bkl,histo,80.0,male,scalp
ISIC_0026769,HAM_0002730,bkl,histo,80.0,male,scalp
ISIC_0025661,HAM_0002730,bkl,histo,80.0,male,scalp
ISIC_0031633,HAM_0001466,bkl,histo,75.0,male,ear
...,...,...,...,...,...,...
ISIC_0033084,HAM_0002867,akiec,histo,40.0,male,abdomen
ISIC_0033550,HAM_0002867,akiec,histo,40.0,male,abdomen
ISIC_0033536,HAM_0002867,akiec,histo,40.0,male,abdomen


#### <font color='#b185db'>Images</font>

Defining the directories for the training and testing images.

In [ ]:
train_images_DIR = "/content/drive/MyDrive/ProjectDL23_24/train"
test_images_DIR = "/content/drive/MyDrive/ProjectDL23_24/test"

The following function, identifies the largest and smallest image in each directory.

In [ ]:
'''# Function to find the largest and smallest images in a directory
def find_extreme_images(directory):
  # Define the variables of smalles and biggest images
  largest_image = None
  smallest_image = None
  largest_size = 0
  smallest_size = float('inf') ## infinit number

  # Iteration loop for each folder to compare the image sizes
  for file in os.listdir(directory):
    file_path = os.path.join(directory, file)
    with Image.open(file_path) as img:
      img_size = img.size[0] * img.size[1]
      # overwrite the variables if bigger/smaller
      if img_size > largest_size:
        largest_size = img_size
        largest_image = file_path, img.size
      if img_size < smallest_size:
        smallest_size = img_size
        smallest_image = file_path, img.size

  return largest_image, smallest_image

# Find the largest and smallest images in train and test directories
largest_train_image, smallest_train_image = find_extreme_images(train_images_DIR)
largest_test_image, smallest_test_image = find_extreme_images(test_images_DIR)

print("Largest image in the train directory:")
print("File Path:", largest_train_image[0])
print("Size (Width x Height):", largest_train_image[1])
print("\nSmallest image in the train directory:")
print("File Path:", smallest_train_image[0])
print("Size (Width x Height):", smallest_train_image[1])
print("\nLargest image in the test directory:")
print("File Path:", largest_test_image[0])
print("Size (Width x Height):", largest_test_image[1])
print("\nSmallest image in the test directory:")
print("File Path:", smallest_test_image[0])
print("Size (Width x Height):", smallest_test_image[1])
'''

'# Function to find the largest and smallest images in a directory\ndef find_extreme_images(directory):\n  # Define the variables of smalles and biggest images\n  largest_image = None\n  smallest_image = None\n  largest_size = 0\n  smallest_size = float(\'inf\') ## infinit number\n\n  # Iteration loop for each folder to compare the image sizes\n  for file in os.listdir(directory):\n    file_path = os.path.join(directory, file)\n    with Image.open(file_path) as img:\n      img_size = img.size[0] * img.size[1]\n      # overwrite the variables if bigger/smaller\n      if img_size > largest_size:\n        largest_size = img_size\n        largest_image = file_path, img.size\n      if img_size < smallest_size:\n        smallest_size = img_size\n        smallest_image = file_path, img.size\n\n  return largest_image, smallest_image\n\n# Find the largest and smallest images in train and test directories\nlargest_train_image, smallest_train_image = find_extreme_images(train_images_DIR)\nlargest

Largest image in the train directory:
File Path: /content/drive/MyDrive/ProjectDL23_24/train/ISIC_0029682.jpg
Size (Width x Height): (600, 450)

Smallest image in the train directory:
File Path: /content/drive/MyDrive/ProjectDL23_24/train/ISIC_0029682.jpg
Size (Width x Height): (600, 450)

Largest image in the test directory:
File Path: /content/drive/MyDrive/ProjectDL23_24/test/ISIC_0026883.jpg
Size (Width x Height): (600, 450)

Smallest image in the test directory:
File Path: /content/drive/MyDrive/ProjectDL23_24/test/ISIC_0026883.jpg
Size (Width x Height): (600, 450)

<b>All Images have the same size</b>

<br>

The following function resizes the images to 50x50. We chose such small numbers to not have problems with the ram capacity.

In [ ]:
def resize_and_create_arrays(directory):
  ## Define the X to store the arrays of images
  X=[]
  ## Define Y to store the labels of the images
  y=[]
  for img in tqdm(os.listdir(directory)):
    path = os.path.join(directory, img)
    # Get label and append to list
    y.append(metadata_df.at[img[:-4], 'dx'])
    # Apply image transformations
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    # Resize because of RAM
    img = cv2.resize(img, (50, 50))
    # Append to list
    X.append(np.array(img))
  # Return X and y
  return X, y

Put variables with the same name besause of RAM

In [ ]:
# Resize images and create X and y arrays for train directory
X_train, y_train = resize_and_create_arrays(train_images_DIR)

100%|██████████| 7511/7511 [07:55<00:00, 15.79it/s]


In [ ]:
# Resize images and create X and y arrays for test directory
X_test, y_test = resize_and_create_arrays(test_images_DIR)

100%|██████████| 2504/2504 [01:40<00:00, 24.84it/s]


The code below converts the images into a NumPy array, in order to be compatible with the neural network model.

In [ ]:
# Convert X and y to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
# Print the shape of the resulting arrays
print("Resized images and created X and y arrays.")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

Resized images and created X and y arrays.
X_train shape: (7511, 50, 50, 3)
y_train shape: (7511,)
X_test shape: (2504, 50, 50, 3)
y_test shape: (2504,)


#### <font color='#b185db'>Label Encoding</font>

Since our target variable is numerical we need to transform it into a numerical format to be able to feed it to the neural network.

In [ ]:
# Convert the labels into a numerical format
labelEncoder = LabelEncoder()
y_train = labelEncoder.fit_transform(y_train)
y_test = labelEncoder.transform(y_test)

# Resclaing of pixels
X_train = X_train/255
X_test = X_test/255



<br>

### <font color='#b185db'>Grid search</font>

In order to get the best results we used a Grid Search to hyperparameter tuning, using the code that was taken from the pratical class 11. Given the fact that we had run this code multiple times, the output provided was not the one used in the last model and that was the one that gave us the best results. However, we decided to keep this in order to show the steps made.

In [ ]:
def build_model(hp):
    """
    Builds model and sets up hyperparameter space to search.

    Parameters
    ----------
    hp : HyperParameter object
        Configures hyperparameters to tune.

    Returns
    -------
    model : keras model
        Compiled model with hyperparameters to tune.
    """
    # Initialize sequential API and start building model.
    model = keras.Sequential()

    for i in range(1, hp.Int("conv_num_layers", 2, 6)):
        model.add(Conv2D(filters=hp.Int("conv_units_" + str(i), min_value=32, max_value=512, step=32),
                         kernel_size=(2, 2), padding='same', activation='relu'))
        model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())

    # Tune the number of dense layers and units in each.
    # Number of Dense layers: 2 - 6
    # Number of Units: 32 - 512 with step size of 32
    for i in range(1, hp.Int("den_num_layers", 2, 6)):
        model.add(
            keras.layers.Dense(
                units=hp.Int("den_units_" + str(i), min_value=32, max_value=512, step=32),
                activation="relu")
        )
        # Dropout following the previous dense layer means that after each layer, we would have a dropout
        # Tune dropout layer with values from 0 - 0.3 with a step size of 0.1.
        model.add(keras.layers.Dropout(hp.Float("dropout_" + str(i), 0, 0.5, step=0.1)))

    # Add output layer.
    model.add(keras.layers.Dense(units=7, activation="softmax"))

    # Tune learning rate for Adam optimizer with values from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])

    # Define optimizer, loss, and metrics
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss=keras.losses.SparseCategoricalCrossentropy(),
                  metrics=["accuracy"])

    return model

In [ ]:
# Instantiate the tuner
tuner = kt.Hyperband(build_model, # the hypermodel
                     objective='val_accuracy',  # Our main monitoring metric
                     max_epochs=10,
                     factor=3, # factor which you have seen above
                     directory='/content', # directory to save logs
                     project_name='try01') # You can name the project itself

In [ ]:
tuner.search(X_train, y_train, epochs=20, validation_split=0.2)

Trial 21 Complete [00h 26m 26s]
val_accuracy: 0.7451763153076172

Best val_accuracy So Far: 0.7797737717628479
Total elapsed time: 05h 31m 15s

Search: Running Trial #22

Value             |Best Value So Far |Hyperparameter
2                 |4                 |conv_num_layers
320               |32                |conv_units_1
6                 |2                 |den_num_layers
512               |128               |den_units_1
0.3               |0.3               |dropout_1
0.001             |0.001             |learning_rate
96                |352               |conv_units_2
448               |160               |conv_units_3
352               |224               |den_units_2
0.2               |0                 |dropout_2
160               |128               |den_units_3
0.4               |0.1               |dropout_3
192               |512               |den_units_4
0.4               |0.4               |dropout_4
128               |192               |den_units_5
0                 |0  

KeyboardInterrupt: ignored

In [ ]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# tuner.search(X_train, y_train, epochs=epochs_no, validation_split=0.2, callbacks=[stop_early])

In [ ]:
best_hp=tuner.get_best_hyperparameters()[0]
print(best_hp.values)
best_model= tuner.hypermodel.build(best_hp)
best_model.fit(X_train, y_train, epochs=20, validation_split=0.2)

{'conv_num_layers': 4, 'conv_units_1': 32, 'den_num_layers': 2, 'den_units_1': 128, 'dropout_1': 0.30000000000000004, 'learning_rate': 0.001, 'conv_units_2': 352, 'conv_units_3': 160, 'den_units_2': 224, 'dropout_2': 0.0, 'den_units_3': 128, 'dropout_3': 0.1, 'den_units_4': 512, 'dropout_4': 0.4, 'den_units_5': 192, 'dropout_5': 0.0, 'conv_units_4': 384, 'conv_units_5': 96, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
Epoch 1/20
188/188 [==============================] - 101s 489ms/step - loss: 1.0536 - accuracy: 0.6641 - val_loss: 0.9258 - val_accuracy: 0.6820
Epoch 2/20
188/188 [==============================] - 75s 397ms/step - loss: 0.9334 - accuracy: 0.6709 - val_loss: 0.8902 - val_accuracy: 0.7152
Epoch 3/20
188/188 [==============================] - 75s 398ms/step - loss: 0.8582 - accuracy: 0.6813 - val_loss: 0.7987 - val_accuracy: 0.7332
Epoch 4/20
188/188 [==============================] - 76s 404ms/step - loss: 

In [ ]:
y_test_predicted = model.predict(X_test)
y_test_predicted= np.argmax(y_test_predicted,axis=1)
print(classification_report(y_test, y_test_predicted))

79/79 [==============================] - 10s 127ms/step
              precision    recall  f1-score   support

           0       0.03      1.00      0.06        82
           1       0.00      0.00      0.00       129
           2       0.00      0.00      0.00       275
           3       0.00      0.00      0.00        29
           4       0.00      0.00      0.00       278
           5       0.00      0.00      0.00      1676
           6       0.00      0.00      0.00        35

    accuracy                           0.03      2504
   macro avg       0.00      0.14      0.01      2504
weighted avg       0.00      0.03      0.00      2504



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<br>

## Performing cross validation of the built model on the data with the same parameters

In [ ]:
earl_stop= EarlyStopping(monitor="loss", patience=3)

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []
history_per_fold = []

# Define the Cross Validator
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Cross Validation model evaluation
fold_no = 1
for train, test in skf.split(X_train, y_train):

    # Best model from grid search
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=352, kernel_size=(2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(filters=160, kernel_size=(2, 2), padding='same', activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(units=128, activation="relu"))
    model.add(Dropout(0.5))  # Fix: Use a dropout rate like 0.5
    model.add(Dense(units=7, activation="softmax"))
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Fit data to model and store the losses per fold during training
    history = model.fit(X_train[train], y_train[train],
                        batch_size=32,
                        epochs=20,
                        verbose=1,
                        callbacks=[earl_stop])
    history_per_fold.append(history.history['loss'])

    # Generate generalization metrics and evaluate on the k-1 fold
    scores = model.evaluate(X_train[test], y_train[test], verbose=1)  # Fix: Use X_train and y_train for evaluation
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])

    # Increase fold number
    fold_no += 1

# Plot the loss for each fold in respect to the number of epochs it has trained
# Define the figure and axes
fig, axes = plt.subplots(2, 2, figsize=(10, 10))

# Plot the loss for each fold
for fold, ax in enumerate(axes.flat):
    ax.plot(range(len(history_per_fold[fold])), history_per_fold[fold], label=f'Fold {fold+1}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.legend()

# Tighten the layout
fig.tight_layout()

# Save the figure as a single image with four subplots
plt.savefig('training_loss_per_fold.png')
plt.show()

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/20
188/188 [==============================] - 73s 381ms/step - loss: 1.0765 - accuracy: 0.6628
Epoch 2/20
188/188 [==============================] - 78s 417ms/step - loss: 0.9316 - accuracy: 0.6743
Epoch 3/20
188/188 [==============================] - 68s 364ms/step - loss: 0.8726 - accuracy: 0.6803
Epoch 4/20
188/188 [==============================] - 78s 414ms/step - loss: 0.8450 - accuracy: 0.6833
Epoch 5/20
188/188 [==============================] - 74s 393ms/step - loss: 0.8098 - accuracy: 0.7004
Epoch 6/20
188/188 [==============================] - 69s 367ms/step - loss: 0.7776 - accuracy: 0.7106
Epoch 7/20
188/188 [==============================] - 74s 392ms/step - loss: 0.7527 - accuracy: 0.7219
Epoch 8/20
188/188 [==============================] - 76s 405ms/step - loss: 0.7367 - accuracy: 0.7197
Epoch 9/20
188/188 [==============================] - 65s 345ms/step - loss: 0.

In [ ]:
#Provide average scores
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - MAE: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> MAE: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')